# Data Preprocessing YTX Step 1

#### Yellow Taxi Data Dictionary
Taken from : __[YTX data dictionary](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)__
1. __*VendorID*__ : A code indicating the TPEP provider that provided the record.
    * 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
1. __*tpep_pickup_datetime*__ : The date and time when the meter was engaged.
1. __*tpep_dropoff_datetime*__ : The date and time when the meter was disengaged.
1. __*Passenger_count*__ : The number of passengers in the vehicle.
    * This is a driver-entered value.
1. __*Trip_distance*__ : The elapsed trip distance in miles reported by the taximeter.
1. __*RateCodeID*__ : The final rate code in effect at the end of the trip.
    * 1 = Standard rate, 2 = JFK, 3 = Newark, 4 = Nassau or Westchester, 5 = Negotiated fare, 6 = Group ride
1. __*Store_and_fwd_flag*__ : This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server.
    * Y = 1.0 = store and forward trip, N = 0.0 = not a store and forward trip
1. __*PULocationID*__ : Pickup Location ID where the meter was engaged. 
1. __*DOLocationID*__ : Dropoff Location ID where the meter was disengaged.
1. __*Payment_type*__ : A numeric code signifying how the passenger paid for the trip.
    * 1 = Credit card, 2 = Cash, 3 = No charge, 4 = Dispute, 5 = Unknown, 6 = Voided trip
1. __*Fare_amount*__ : The time-and-distance fare calculated by the meter.
1. __*Extra*__ : Miscellaneous extras and surcharges. Currently, this only includes the \\$0.50 and \\$1 rush hour and overnight charges.
1. __*MTA_tax*__ : \\$0.50 MTA tax that is automatically triggered based on the metered rate in use.
1. __*Improvement_surcharge*__ : \\$0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
1. __*Tip_amount*__ : Tip amount – This field is automatically populated for credit card tips. Cash tips are not included.
1. __*Tolls_amount*__ : Total amount of all tolls paid in trip.
1. __*Total_amount*__ : The total amount charged to passengers. Does not include cash tips.

___

In [1]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *

import warnings
warnings.filterwarnings("ignore")

#sc = SparkContext.getOrCreate(conf=swan_spark_conf) 
spark = SparkSession.builder.appName('Data Preprocessing Step 1').getOrCreate()
spark

In [2]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

### Yellow Taxi (2018)
#### Setting Schema

In [3]:
ytx_2018_sdf = spark.read.csv('../raw_data/YTX/2018', header=True)

In [4]:
ints = ('VendorID', 'passenger_count', 'RateCodeID', 'RatecodeID'
        ,'payment_type', 'PULocationID', 'DOLocationID')

doubles = ('trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 
           'tolls_amount', 'improvement_surcharge', 'total_amount')

strings = ('store_and_fwd_flag',)

dtimes = ('tpep_pickup_datetime', 'tpep_dropoff_datetime', )

dtypes = {column: IntegerType() for column in ints}
dtypes.update({column: DoubleType() for column in doubles})
dtypes.update({column: StringType() for column in strings})
dtypes.update({column: TimestampType() for column in dtimes})

In [5]:
schema = StructType()

for column in ytx_2018_sdf.columns:
    schema.add(column, # column name
               dtypes[column], # data type
               True # is nullable?
              )

In [6]:
ytx_2018_sdf = spark.read.csv('../raw_data/YTX/2018', header = True, schema = schema) \
    .withColumnRenamed("RatecodeID","RateCodeID") # rename the wrong column

ytx_2018_sdf.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [7]:
from pyspark.sql.functions import isnan, when, count, col

ytx_2018_sdf.select([count(when(col(c).isNull(), c)).alias(c) for c in ytx_2018_sdf.columns]).limit(1)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RateCodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


> No missing values was found!

In [8]:
ytx_2018_sdf.limit(5)
original_count = ytx_2018_sdf.count()

> As we see from the dictionary that `store_and_fwd_flag` can be said to be boolean type (Yes or No), which we will convert here to True and False as it is currently in string type.

In [9]:
ytx_2018_sdf = ytx_2018_sdf.withColumn("store_and_fwd_flag", 
                                             (ytx_2018_sdf["store_and_fwd_flag"] == 'Y') \
                                             .cast("boolean"))

In [10]:
ytx_2018_sdf.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [11]:
ytx_2018_sdf.describe()

summary,VendorID,passenger_count,trip_distance,RateCodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,102804250,102804250,102804250,102804250,102804250,102804250,102804250,102804250,102804250,102804250,102804250,102804250,102804250,102804250
mean,1.5917952419282277,1.5949764333672976,2.935368240515281,1.0501113718547628,163.1976826833521,161.42923143741626,1.3139463300398573,13.060032438153184,0.33070590924013327,0.49713522106333163,1.8747264632543357,0.34687786127552644,0.299587523659946,16.412771868007482
stddev,0.5198787068970184,1.2412303351510654,19.126793016573206,0.7578954227329817,66.53010479123175,70.41625751123695,0.4844298740802132,150.83157884972647,0.4660741653145101,0.05022877093569929,2.6117606925611883,1.7916384543455615,0.4933835174627733,151.07475384454816
min,1,0,0.0,1,1,1,1,-800.0,-80.0,-0.5,-322.42,-52.5,-0.3,-800.3
max,4,192,189483.84,99,265,265,5,907070.24,96.64,150.0,945.97,1650.0,4000.3,907071.04


> Based on the description/distribution of the dataset, it looks like `fare_amount`, `extra`, `mta_tax`, `tip_amount`, `tolls_amount`, `improvement_surcharge` and `total_amount` have __negative__ values. This does not make any sense, it could mean a lot of thing as it does not mean necessarily means missing values or error. Just what is the underlying reason behind this negative values? Anyhow, we cannot remove it right now as not there is not enough evidence.
---
---

### Cleaning
> VendorID should only be 1 and 2, but here the maximum is 4.

> Inaccurate pickup and dropoff time if pickup happen after dropoff.

> In RateCodeID, we also see that the value should always be between 1 and 6 but it has a maximum of 99.

> Fare amount less than 2.5 should only be payment type 3 to 6.

> The `fare_amount` should be more than the minimum fare which is 0.5\\$ per 1/5 mile or per min and less than the maximum fare stated.

> According to the dictionary, `extra` should only include \\$0.5 rush hour surcharge and \\$1.0 overnight surcharge. But in this case, it has a maximum of 96.64 and minimum of -80.0.

> mta_tax should only be \$0.5 surcharge, but it has a maximum of 150.

> tip_amount should only be related to payment_type 1 (credit).

> improvement_surcharge should only be \$0.3 as well, but here we can see the maximum is 4000.3.

> We also see irregularities in `passenger_count` which have maximum of 192. The maximum number of passengers is 6 as stated.



#### Calculating Trip Duration

In [12]:
# Adding trip duration in minutes by rounding the difference of dropoff and pickup time.
ytx_2018_sdf = ytx_2018_sdf.withColumn('trip_duration', 
                        f.round((ytx_2018_sdf['tpep_dropoff_datetime'].cast('long') \
                                 - ytx_2018_sdf['tpep_pickup_datetime'].cast('long')) / 60))

ytx_2018_sdf = ytx_2018_sdf.filter('trip_duration > 0')

#### Computing Average Speed
> The speed limit statewide in NYC was __changed to 25 mph from 30 mph in 7th November of 2014__[1], unless otherwise signed as part of __Vision Zero initiative__ conducted by NYC government. Only 3% of those previous speed limit (30 mph) was left. Furthermore, out of those 3%, 25% of it are on Staten Island and 48% are located in Queens. We also want to remove those records that are speeding as the __fine ranges from \\$90 to \\$600 for first conviction__[3] and would not be profitable. As we are aiming for profitability of Yellow Taxis, we should maintain the maximum speed limit to be 25-30 mph without exceeding it at all as perhaps some passenger might ask the driver to went over the speed limit a bit.

>https://www1.nyc.gov/html/dot/html/motorist/motorist.shtml

> Mapping : https://data.cityofnewyork.us/Transportation/VZV_Speed-Limits/7n5j-865y

>https://trafficsafety.ny.gov/penalties-speeding

In [13]:
# Drop any 0 trip duration as it may means the taxi is not moving or the record itself is inaccurate
ytx_2018_sdf = ytx_2018_sdf.filter('trip_distance > 0')

In [14]:
# Calculating average speed over the trip distance over the duration.
ytx_2018_sdf = ytx_2018_sdf.withColumn('avg_speed', 
                        f.round(ytx_2018_sdf['trip_distance']/(ytx_2018_sdf['trip_duration']/60), 2))

ytx_2018_sdf = ytx_2018_sdf.filter('avg_speed <= 30')

#### VendorID

In [15]:
ytx_2018_sdf = ytx_2018_sdf.filter('VendorID in (1, 2)')

#### Inaccurate Passenger Count

> Based on few description above, we can see that the __minimum__ of `passenger_count` is __zero__ and the maximum are __192__. Furthermore, we assume that __zero__ `passenger_count` to inaccurate or not recorded properly to be removed. The removal may be justified with reasoning that it may hinder in our analysis as we aim to maximise profitability with of course passenger riding the taxi. The question for this is that __"How can a taxi with no passenger riding can gain profit? is it reasonable?"__.

> Maybe someone ask a taxi driver to deliver some stuffs?

> From Driver Rule 54-15(g) Chapter 54 - Drivers of Taxicabs and Street Hail Liveries. __"Overloading Vehicle. A Driver must not permit more than four Passengers to
ride in a four-Passenger Vehicle, nor more than five Passengers in a fivePassenger Vehicle, except that an additional Passenger must be accepted if the
Passenger is under the age of seven (7) and is held on the lap of an adult
Passenger seated in the rear. A Driver must not permit a Street Hail Livery to be
overloaded or to carry more passengers than the Vehicle is equipped to seat based
on the NYS DMV registration of the Vehicle."__

> https://www1.nyc.gov/assets/tlc/downloads/pdf/rule_book_current_chapter_54.pdf

> With that Rule stated, we can say that the maximum passengers on taxi-cab are 5 adults and 1 child under the age of seven. Therefore, __maximum of 6 passengers__ can be allowed.

In [16]:
ytx_2018_sdf = ytx_2018_sdf.filter('passenger_count >= 0 and passenger_count <= 6')

#### Improper RateCodeID

In [17]:
ytx_2018_sdf = ytx_2018_sdf.filter('RateCodeID <= 6')

#### Fare Amount less than $2.5

In [18]:
ytx_2018_sdf.filter(ytx_2018_sdf['fare_amount'] < 2.5).limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RateCodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration,avg_speed
2,2018-01-01 01:02:39,2018-01-01 01:04:02,2,0.09,1,false,87,87,4,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,1.0,5.4
2,2018-01-01 00:38:19,2018-01-01 00:55:57,1,0.41,1,false,68,68,4,-11.0,-0.5,-0.5,0.0,0.0,-0.3,-12.3,18.0,1.37
2,2018-01-01 00:47:40,2018-01-01 00:49:06,1,0.13,1,false,137,137,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,1.0,7.8
2,2018-01-01 00:11:10,2018-01-01 00:15:14,1,0.16,1,false,137,170,4,-4.5,-0.5,-0.5,0.0,0.0,-0.3,-5.8,4.0,2.4
2,2018-01-01 00:22:25,2018-01-01 00:26:42,2,0.12,1,false,48,48,3,-4.5,-0.5,-0.5,0.0,0.0,-0.3,-5.8,4.0,1.8


> It looks like that the negative `fare_amount`, `extra`, `mta_tax`, `tip_amount`, `tolls_amount`, `improvement_surcharge` and `total_amount` may be caused by payment type 3 (No Charge) and payment type 4 (Dispute). Although, we see that `fare_amount` can be negative so it should be less than 0, but the incorrect records for it should be less than \\$2.5 as the __initial charge of a trip is always \\$2.5__. But is it all caused by payment type 3 and type 4?

In [19]:
ytx_2018_sdf.select('payment_type', 'fare_amount') \
            .filter(ytx_2018_sdf['fare_amount'] < 2.5) \
            .groupBy('payment_type').count()

payment_type,count
1,1263
3,33712
4,16936
2,7241


> Here we can say that `payment_type` 1 and 2 with `fare_amount` __less than \\$2.5__ to be inaccurate records. Therefore, we can exclude those rows from the dataset.

In [20]:
ytx_2018_sdf = ytx_2018_sdf.filter("fare_amount >= 2.5 or (fare_amount < 2.5 and (payment_type not in (1, 2)))")

#### Inaccurate Fare Amount
> Next, we want to check if the `fare_amount` were calculated accurately, with $2.50 initial charge
Plus 50 cents per 1/5 mile when traveling above 12mph or per 60 seconds in slow traffic or when the vehicle is stopped.

> According to the TLC website, the `fare_amount` can be decluttered as such: 
<center>
$ f(n) =
  \begin{cases}
    \$2.5 \text{ per mile}      & \quad \text{if speed} > 12 \text{ mph}\\
    \$2.5 \text{ per mile} =  \$0.5 \text{ per minute} & \quad \text{if speed} = 12 \text{ mph} \\
    \$0.5 \text{ per minute}      & \quad \text{if speed} < 12 \text{ mph}
  \end{cases}
$
</center>

> With that, we can derived the __minimum__ of riding a Yellow Taxi as such:
<center>
$ minimum fare = \$2.5 + max(0.5 \cdot minute, 2.5 \cdot distance)$
</center>

> We also want the negative `fare_amount` to be calculated as such, therefore we have to take the absolute of the fare amount. <center>$\text{fare per minute or fare per 1/5 mile} = \frac{|\text{fare amount}| - 2.5}{max(0.5 \cdot minute, 2.5 \cdot miles)}$

> https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page 
    
> https://puzzling.stackexchange.com/questions/101441/the-range-of-taxi-fares

In [21]:
# minimal fare = 2.5 + max(0.5 * minute, 2.5 * distance)
ytx_2018_sdf = ytx_2018_sdf.withColumn('min_fare', 2.5 + \
                        f.greatest(0.5*ytx_2018_sdf['trip_duration'], 
                                   2.5*ytx_2018_sdf['trip_distance']))

In [22]:
ytx_2018_sdf = ytx_2018_sdf.filter(f.abs(ytx_2018_sdf['fare_amount']) >= ytx_2018_sdf['min_fare'])

#### Inaccurate extra, mta_tax and improvement_surcharge records

In [23]:
# Filtering out the irregularities found in extra, mta_tax and improvement_surcharge according
# to the data dictionary.
ytx_2018_sdf = ytx_2018_sdf.filter('extra in (-1, -0.5, 0.5, 1)') \
                           .filter('mta_tax in (-0.5, 0.5)') \
                           .filter('improvement_surcharge in (-0.3, 0.3)')

#### Inaccurate tip amount

In [24]:
ytx_2018_sdf = ytx_2018_sdf.filter(((f.abs(ytx_2018_sdf['tip_amount']) >= 0) & (ytx_2018_sdf['payment_type'] == 1)) |
                                    (ytx_2018_sdf['tip_amount'] == 0) & (ytx_2018_sdf['payment_type'] != 1))

#### Payment Type

In [25]:
ytx_2018_sdf = ytx_2018_sdf.filter('payment_type = 1')

#### PULocationID and DOLocationID

In [26]:
# In taxi zone lookup location starts from id 1 and (id 264 and 264) are unknown.
ytx_2018_sdf = ytx_2018_sdf.filter('PULocationID > 0 and PULocationID <= 263') \
                           .filter('PULocationID > 0 and PULocationID <= 263')

### Dropping Irrelevant features

In [27]:
ytx_2018_sdf.describe()

summary,VendorID,passenger_count,trip_distance,RateCodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration,avg_speed,min_fare
count,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260,31456260
mean,1.581947313507709,1.5976310915537957,2.722455032480016,1.000926556431057,160.4036136209454,157.66923982698515,1.0,12.046803216911355,0.6783010281578293,0.5,2.4881992493701777,0.1890320969498349,0.30000000000117766,16.206657939908666,13.690371360104475,11.505722613240748,10.530696982095087
stddev,0.49323893352302384,1.2291365682815243,2.647296313363658,0.05271382450874234,64.81510185927299,72.16645926434423,0.0,8.003740709372236,0.23949852895543547,0.0,2.0173977230752254,1.054756009528139,0.0,10.031378994962974,9.66287788595142,4.878551397082311,6.673947520663254
min,1,0,0.01,1,1,1,1,3.0,-0.5,0.5,0.0,0.0,0.3,4.3,1.0,0.01,3.0
max,2,6,89.57,4,263,265,1,411.5,1.0,0.5,457.11,189.99,0.3,469.41,592.0,30.0,298.5


In [28]:
ytx_2018_sdf = ytx_2018_sdf.drop('payment_type', 'VendorID', 'min_fare', 'store_and_fwd_flag', 'total_amount',
                                 'extra', 'MTA_tax', 'improvement_surcharge', 'passenger_count', 'RateCodeID')

#### To parquet

In [29]:
# Check to see if the fpath already exists. If so, remove it.
from shutil import rmtree
from os import path

fpath = '../preprocessed_data/ytx_2018_cleaned.parquet/'

if path.exists(fpath):
    rmtree(fpath)
        
ytx_2018_sdf.write.parquet('../preprocessed_data/ytx_2018_cleaned.parquet')

In [30]:
temp = spark.read.parquet('../preprocessed_data/ytx_2018_cleaned.parquet/')
reduced_count = temp.count()
print("Original count:", original_count)
print(f"Reduced by: {original_count - reduced_count}")
print("Reduced by (%):", "{:.4f}".format((original_count - reduced_count) / original_count))
print("Reduced Count:", reduced_count)

Original count: 102804250
Reduced by: 71347990
Reduced by (%): 0.6940
Reduced Count: 31456260


### Yellow Taxi 2019

In [31]:
ytx_2019_sdf = spark.read.csv('../raw_data/YTX/2019', header=True)

In [32]:
ints = ('VendorID', 'passenger_count', 'RateCodeID', 'RatecodeID'
        ,'payment_type', 'PULocationID', 'DOLocationID')

doubles = ('trip_distance', 'fare_amount', 'extra', 'mta_tax', 
           'tip_amount', 'tolls_amount', 'improvement_surcharge', 
           'total_amount', 'congestion_surcharge')

strings = ('store_and_fwd_flag',)

dtimes = ('tpep_pickup_datetime', 'tpep_dropoff_datetime', )

dtypes = {column: IntegerType() for column in ints}
dtypes.update({column: DoubleType() for column in doubles})
dtypes.update({column: StringType() for column in strings})
dtypes.update({column: TimestampType() for column in dtimes})

schema = StructType()

for column in ytx_2019_sdf.columns:
    schema.add(column, # column name
               dtypes[column], # data type
               True # is nullable?
              )

In [33]:
ytx_2019_sdf = spark.read.csv('../raw_data/YTX/2019', header = True, schema = schema) \
    .withColumnRenamed("RatecodeID","RateCodeID") # rename the wrong column

ytx_2019_sdf.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [34]:
from pyspark.sql.functions import isnan, when, count, col

ytx_2019_sdf.select([count(when(col(c).isNull(), c)).alias(c) for c in ytx_2019_sdf.columns]).limit(1)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RateCodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
246601,0,0,246601,0,246601,246601,0,0,246601,0,0,0,0,0,0,0,4855981


> Here we can see that there is missing values, which we going to drop as this will be the prediction set and considered as unseen (only cleaned)

In [35]:
original_count = ytx_2019_sdf.count()
ytx_2019_sdf = ytx_2019_sdf.dropna(how = 'any')

In [36]:
ytx_2019_sdf = ytx_2019_sdf.withColumn("store_and_fwd_flag", 
                                             (ytx_2019_sdf["store_and_fwd_flag"] == 'Y') \
                                             .cast("boolean"))

In [37]:
# Making sure store_and_fwd_flag has been set to boolean.
ytx_2019_sdf.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [38]:
# Adding trip duration in minutes by rounding the difference of dropoff and pickup time.
ytx_2019_sdf = ytx_2019_sdf.withColumn('trip_duration', 
                        f.round((ytx_2019_sdf['tpep_dropoff_datetime'].cast('long') \
                                 - ytx_2019_sdf['tpep_pickup_datetime'].cast('long')) / 60))

ytx_2019_sdf = ytx_2019_sdf.filter('trip_duration > 0')

In [39]:
# Drop any 0 trip duration as it may means the taxi is not moving or the record itself is inaccurate
ytx_2019_sdf = ytx_2019_sdf.filter('trip_distance > 0')

In [40]:
# Calculating average speed over the trip distance over the duration.
ytx_2019_sdf = ytx_2019_sdf.withColumn('avg_speed', 
                        f.round(ytx_2019_sdf['trip_distance']/(ytx_2019_sdf['trip_duration']/60), 2))

ytx_2019_sdf = ytx_2019_sdf.filter('avg_speed <= 30')

In [41]:
# Follows the 2018 dataset.
# VendorID are only 1 and 2 according to dictionary
# Maximum legal passenger count of yellow taxi cab are 6
# RateCodeID available are only 1, 2, 3, 4, 5 and 6
# fare_amount that are negative can only be other than payment type 1 and 2

ytx_2019_sdf = ytx_2019_sdf.filter('VendorID in (1, 2)') \
                           .filter('passenger_count >= 0 and passenger_count <= 6') \
                           .filter('RateCodeID <= 6') \
                           .filter("fare_amount >= 2.5 or (fare_amount < 2.5 and (payment_type not in (1, 2)))")

In [42]:
# minimal fare = 2.5 + max(0.5 * minute, 2.5 * distance)
ytx_2019_sdf = ytx_2019_sdf.withColumn('min_fare', 2.5 + \
                        f.greatest(0.5*ytx_2019_sdf['trip_duration'], 
                                   2.5*ytx_2019_sdf['trip_distance']))

In [43]:
# Inaccurate extra, mta_tax and improvement_surcharge
ytx_2019_sdf = ytx_2019_sdf.filter('extra in (-1, -0.5, 0.5, 1)') \
                           .filter('mta_tax in (-0.5, 0.5)') \
                           .filter('improvement_surcharge in (-0.3, 0.3)')

In [44]:
# Tip amount can only be charged with payment type 1
ytx_2019_sdf = ytx_2019_sdf.filter(((f.abs(ytx_2019_sdf['tip_amount']) >= 0) & (ytx_2019_sdf['payment_type'] == 1)) |
                                    (ytx_2019_sdf['tip_amount'] == 0) & (ytx_2019_sdf['payment_type'] != 1))

In [45]:
ytx_2019_sdf = ytx_2019_sdf.filter('payment_type = 1')

In [46]:
# In taxi zone lookup location starts from id 1 and (id 264 and 264) are unknown.
ytx_2019_sdf = ytx_2019_sdf.filter('PULocationID > 0 and PULocationID <= 263') \
                           .filter('PULocationID > 0 and PULocationID <= 263')

In [47]:
ytx_2019_sdf = ytx_2019_sdf.drop('payment_type', 'VendorID', 'min_fare', 'store_and_fwd_flag', 'total_amount',
                                 'extra', 'MTA_tax', 'improvement_surcharge', 'passenger_count', 'RateCodeID',
                                 'congestion_surcharge')

In [48]:
from shutil import rmtree
from os import path

fpath = '../preprocessed_data/ytx_2019_cleaned.parquet/'
if path.exists(fpath):
    rmtree(fpath)
ytx_2019_sdf.write.format('parquet').save('../preprocessed_data/ytx_2019_cleaned.parquet')    

In [49]:
temp = spark.read.parquet('../preprocessed_data/ytx_2019_cleaned.parquet/')
reduced_count = temp.count()
print("Original count:", original_count)
print(f"Reduced by: {original_count - reduced_count}")
print("Reduced by (%):", "{:.4f}".format((original_count - reduced_count) / original_count))
print("After Reduction:", reduced_count)

Original count: 84399019
Reduced by: 66938569
Reduced by (%): 0.7931
After Reduction: 17460450
